In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="Iris"
JobNameAbbrev = "M1"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list([0, 7, 10, 19, 30, 49, 50, 59, 64, 76, 83]),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "rashomon_bound_adder": [0.041, 0.043, 0.045],
                       "Type": ["Classification"]}

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Random Forest Simulations

In [3]:
# ### Include Random Forest ###
# RandomForestParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["TreeEnsembleQBCFunction"],
#                        "ModelType":["RandomForestClassificationFunction"],
#                        "UniqueErrorsInput": [0],
#                        "n_estimators": [100], 
#                        "regularization": [0.00],
#                        "rashomon_bound_adder": [0],
#                        "Type": ["Classification"]}
# RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# # NOTE: Comment out chunk to not include random forest simulations. ###
# ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
# ParameterVector = ParameterVector.sort_values("Seed")
# ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [4]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["rashomon_bound_adder"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_DPL", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_UNQ", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True))

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

# Save Parameter Vector

In [5]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,rashomon_bound_adder,Type,JobName,Output
0,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.041,Classification,0M1_DPL0.041,Iris/TreeFarms/Raw/0M1_DPL0.041.pkl
1,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.043,Classification,0M1_DPL0.043,Iris/TreeFarms/Raw/0M1_DPL0.043.pkl
2,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.045,Classification,0M1_DPL0.045,Iris/TreeFarms/Raw/0M1_DPL0.045.pkl
3,Iris,7,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.041,Classification,7M1_DPL0.041,Iris/TreeFarms/Raw/7M1_DPL0.041.pkl
4,Iris,7,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.043,Classification,7M1_DPL0.043,Iris/TreeFarms/Raw/7M1_DPL0.043.pkl
5,Iris,7,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.045,Classification,7M1_DPL0.045,Iris/TreeFarms/Raw/7M1_DPL0.045.pkl
6,Iris,10,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.041,Classification,10M1_DPL0.041,Iris/TreeFarms/Raw/10M1_DPL0.041.pkl
7,Iris,10,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.043,Classification,10M1_DPL0.043,Iris/TreeFarms/Raw/10M1_DPL0.043.pkl
8,Iris,10,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.045,Classification,10M1_DPL0.045,Iris/TreeFarms/Raw/10M1_DPL0.045.pkl
9,Iris,19,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.041,Classification,19M1_DPL0.041,Iris/TreeFarms/Raw/19M1_DPL0.041.pkl


In [6]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

---

In [7]:
# ### Simulations that failed ###
# FilterText = ['66IS_UNQ0.041', 
# '66IS_UNQ0.043', 
# '66IS_UNQ0.045', 
# '13IS_UNQ0.045', 
# '66IS_DPL0.041', 
# '66IS_DPL0.043', 
# '66IS_DPL0.045', 
# '13IS_DPL0.045']

# ### Rerun these simulations ###
# ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector
# ### Resave ###
# output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
# ParameterVector.to_csv(output_path, index=False)